In [1]:
import mlflow
import pandas as pd
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np

In [5]:
df = pd.read_csv('IMDB.csv')
df.head()

review sentiment
0  Film version of Sandra Bernhard's one-woman of...  negative
1  I switched this on (from cable) on a whim and ...  positive
2  The `plot' of this film contains a few holes y...  negative
3  Some amusing humor, some that falls flat, some...  negative
4  What can you say about this movie? It was not ...  negative

In [3]:
# data preprocessing

# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub(r'\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['review'] = df['review'].apply(lower_case)
        df['review'] = df['review'].apply(remove_stop_words)
        df['review'] = df['review'].apply(removing_numbers)
        df['review'] = df['review'].apply(removing_punctuations)
        df['review'] = df['review'].apply(removing_urls)
        df['review'] = df['review'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

In [6]:
df = normalize_text(df)
df.head()

review sentiment
0  film version sandra bernhard s one woman off b...  negative
1  switched from cable whim treated quite surpris...  positive
2  plot film contains hole could drive massive tr...  negative
3  amusing humor fall flat decent acting quite at...  negative
4  say movie terrible good two day earlier watche...  negative

In [7]:
df['sentiment'].value_counts()

sentiment
negative    517
positive    483
Name: count, dtype: int64

In [8]:
x = df['sentiment'].isin(['positive','negative'])
df = df[x]

In [9]:
df['sentiment'] = df['sentiment'].map({'positive':1, 'negative':0})
df.head()

review  sentiment
0  film version sandra bernhard s one woman off b...          0
1  switched from cable whim treated quite surpris...          1
2  plot film contains hole could drive massive tr...          0
3  amusing humor fall flat decent acting quite at...          0
4  say movie terrible good two day earlier watche...          0

In [10]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [11]:
vectorizer = CountVectorizer(max_features=100)
X = vectorizer.fit_transform(df['review'])
y = df['sentiment']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [13]:
import dagshub

mlflow.set_tracking_uri('https://dagshub.com/chhalotreprakhar00/Capstone-project.mlflow')
dagshub.init(repo_owner='chhalotreprakhar00', repo_name='Capstone-project', mlflow=True)

# mlflow.set_experiment("Logistic Regression Baseline")
mlflow.set_experiment("Logistic Regression Baseline")


❗❗❗ AUTHORIZATION REQUIRED ❗❗❗

/home/pchhalotre/anaconda3/envs/Capstone/lib/python3.13/site-packages/rich/live.py:231: UserWarning: install 
"ipywidgets" for Jupyter support
  warnings.warn('install "ipywidgets" for Jupyter support')



Open the following link in your browser to authorize the client:
https://dagshub.com/login/oauth/authorize?state=1ed85737-090d-4304-b0aa-dc233e994abb&client_id=32b60ba385aa7cecf24046d8195a71c07dd345d9657977863b52e7748e0f0f28&middleman_request_id=c9f8f39fb5012004000294d6a576b723a524cc790b0b5d5edf63458c6af31726




Accessing as chhalotreprakhar00

Initialized MLflow to track repo "chhalotreprakhar00/Capstone-project"

Repository chhalotreprakhar00/Capstone-project initialized!

2025/04/24 23:44:02 INFO mlflow.tracking.fluent: Experiment with name 'Logistic Regression Baseline' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/558e23a4a6194b279784a0873d2f4bb4', creation_time=1745518442135, experiment_id='0', last_update_time=1745518442135, lifecycle_stage='active', name='Logistic Regression Baseline', tags={}>

In [16]:
import mlflow
import logging
import os
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Configure logging
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

logging.info("Starting MLflow run...")

with mlflow.start_run():
    start_time = time.time()
    
    try:
        logging.info("Logging preprocessing parameters...")
        mlflow.log_param("vectorizer", "Bag of Words")
        mlflow.log_param("num_features", 100)
        mlflow.log_param("test_size", 0.25)

        logging.info("Initializing Logistic Regression model...")
        model = LogisticRegression(max_iter=1000)  # Increase max_iter to prevent non-convergence issues

        logging.info("Fitting the model...")
        model.fit(X_train, y_train)
        logging.info("Model training complete.")

        logging.info("Logging model parameters...")
        mlflow.log_param("model", "Logistic Regression")

        logging.info("Making predictions...")
        y_pred = model.predict(X_test)

        logging.info("Calculating evaluation metrics...")
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)

        logging.info("Logging evaluation metrics...")
        mlflow.log_metric("accuracy", accuracy)
        mlflow.log_metric("precision", precision)
        mlflow.log_metric("recall", recall)
        mlflow.log_metric("f1_score", f1)

        logging.info("Saving and logging the model...")
        mlflow.sklearn.log_model(model, "model", input_example=X_train)

        # Log execution time
        end_time = time.time()
        logging.info(f"Model training and logging completed in {end_time - start_time:.2f} seconds.")

        # Save and log the notebook
        # notebook_path = "exp1_baseline_model.ipynb"
        # logging.info("Executing Jupyter Notebook. This may take a while...")
        # os.system(f"jupyter nbconvert --to notebook --execute --inplace {notebook_path}")
        # mlflow.log_artifact(notebook_path)

        # logging.info("Notebook execution and logging complete.")

        # Print the results for verification
        logging.info(f"Accuracy: {accuracy}")
        logging.info(f"Precision: {precision}")
        logging.info(f"Recall: {recall}")
        logging.info(f"F1 Score: {f1}")

    except Exception as e:
        logging.error(f"An error occurred: {e}", exc_info=True)


2025-04-24 23:54:06,304 - INFO - Starting MLflow run...
2025-04-24 23:54:06,692 - INFO - Logging preprocessing parameters...
2025-04-24 23:54:07,884 - INFO - Initializing Logistic Regression model...
2025-04-24 23:54:07,884 - INFO - Fitting the model...
2025-04-24 23:54:07,897 - INFO - Model training complete.
2025-04-24 23:54:07,897 - INFO - Logging model parameters...
2025-04-24 23:54:08,290 - INFO - Making predictions...
2025-04-24 23:54:08,290 - INFO - Calculating evaluation metrics...
2025-04-24 23:54:08,295 - INFO - Logging evaluation metrics...
2025-04-24 23:54:09,966 - INFO - Saving and logging the model...
2025-04-24 23:54:18,977 - INFO - Model training and logging completed in 12.29 seconds.
2025-04-24 23:54:18,977 - INFO - Accuracy: 0.692
2025-04-24 23:54:18,978 - INFO - Precision: 0.6974789915966386
2025-04-24 23:54:18,978 - INFO - Recall: 0.6693548387096774
2025-04-24 23:54:18,978 - INFO - F1 Score: 0.6831275720164609


🏃 View run righteous-hawk-613 at: https://dagshub.com/chhalotreprakhar00/Capstone-project.mlflow/#/experiments/0/runs/ae6f30665df3485987c8aea96ffc889a
🧪 View experiment at: https://dagshub.com/chhalotreprakhar00/Capstone-project.mlflow/#/experiments/0
